In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install qwen-vl-utils
!pip install openai
!pip install flash-attn --no-build-isolation


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ipet60u8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ipet60u8
  Resolved https://github.com/huggingface/transformers to commit 51d732709e5ae424e8fb6c4e58b72057a3e413c2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11363407 sha256=e3dfbf1208bace9a0b4034c8edb31abcefea30ba67bef0188d162481358594c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-3fj2hwzf/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# =========================================================
# 1. Load Qwen 2.5-VL (7 B Instruct)
# =========================================================
import os, json, re, tempfile, torch, cv2, typing as tp
from PIL import Image
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor

MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype=torch.bfloat16,         # Colab GPUs support bfloat16
)
processor = AutoProcessor.from_pretrained(MODEL_ID)

# Handy helper --------------------------------------------------------------
def inference(img_path: str, prompt: str, *, system_prompt="You are a helpful assistant", max_new_tokens=512):
    """Single‑image chat inference. Returns (raw_text, h_px, w_px)."""
    image = Image.open(img_path).convert("RGB")
    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"image": img_path},
            ],
        },
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)

    gen_ids = [outs[len(ins):] for ins, outs in zip(inputs.input_ids, output_ids)]
    output_text = processor.batch_decode(gen_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]

    h_px = int(inputs["image_grid_thw"][0][1] * 14)  # ViT patch to pixel
    w_px = int(inputs["image_grid_thw"][0][2] * 14)
    return output_text, h_px, w_px

def ask_bbox(pil_img: Image.Image, object_desc: str):
    """Return [x1,y1,x2,y2] in original‑frame pixels."""
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as f:
        pil_img.save(f.name)
        tmp_path = f.name

    prompt = (
        f"Locate the {object_desc}. "
        f"Output JSON ONLY like {{\"bbox_2d\":[x1,y1,x2,y2]}}. "
        f"Use absolute pixel coordinates."
    )

    raw_text, h_px, w_px = inference(tmp_path, prompt)
    m = re.search(r"\{[^}]*\"bbox_2d\"[^}]*\}", raw_text)
    if not m:
        raise ValueError(f"No bbox found for '{object_desc}'.\nAnswer was:\n{raw_text}")
    bbox = json.loads(m.group(0))["bbox_2d"]
    if len(bbox) != 4:
        raise ValueError(f"bbox must have 4 coords, got {bbox}")

    x1, y1, x2, y2 = map(float, bbox)
    # If numbers are 0‑1, scale back to returned resized image
    if 0 <= max(x1, y1, x2, y2) <= 1.5:
        x1, x2 = x1 * w_px, x2 * w_px
        y1, y2 = y1 * h_px, y2 * h_px

    # Map resized image → original frame
    orig_w, orig_h = pil_img.size
    scale_x, scale_y = orig_w / w_px, orig_h / h_px
    x1, x2 = x1 * scale_x, x2 * scale_x
    y1, y2 = y1 * scale_y, y2 * scale_y

    x1, x2 = sorted([int(round(x1)), int(round(x2))])
    y1, y2 = sorted([int(round(y1)), int(round(y2))])
    x1, x2 = max(0, x1), min(orig_w - 1, x2)
    y1, y2 = max(0, y1), min(orig_h - 1, y2)
    if x2 <= x1 or y2 <= y1:
        raise ValueError(f"Invalid bbox after scaling: {[x1,y1,x2,y2]}")
    return [x1, y1, x2, y2]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [3]:
# =========================================================
# 2. Utilities: tracker factory (if / elif / else version)
# =========================================================



def _make_tracker(name: str):
    name = name.lower()
    if name == "csrt" and hasattr(cv2, "TrackerCSRT_create"):
        return cv2.TrackerCSRT_create()
    if name == "kcf" and hasattr(cv2, "TrackerKCF_create"):
        return cv2.TrackerKCF_create()
    if name == "mosse" and hasattr(cv2, "TrackerMOSSE_create"):
        return cv2.TrackerMOSSE_create()
    # fallback
    if hasattr(cv2, "TrackerMOSSE_create"):
        print(f"[WARN] '{name}' unavailable, falling back to MOSSE.")
        return cv2.TrackerMOSSE_create()
    raise RuntimeError("No compatible OpenCV tracker found!")

def _make_tracker_MOI(name: str):
    name = name.lower()
    if name == "csrt" and hasattr(cv2, "TrackerCSRT_create"):
        return cv2.legacy.TrackerCSRT_create()
    elif name == "kcf" and hasattr(cv2, "TrackerKCF_create"):
        return cv2.legacy.TrackerKCF_create()
    elif name == "mosse" and hasattr(cv2, "TrackerMOSSE_create"):
        return cv2.legacy.TrackerMOSSE_create()
    # fallback
    if hasattr(cv2, "TrackerMOSSE_create"):
        print(f"[WARN] '{name}' unavailable, falling back to MOSSE.")
        return cv2.legacy.TrackerMOSSE_create()
    raise RuntimeError("No compatible OpenCV tracker found!")

def track_video(video_path: str,
                object_descs: tp.List[str] | str,
                *,
                tracker_name: str = "csrt",
                out_path: str = "tracked.mp4",
                show_every: int = 0):
    """Track multiple objects described by natural language prompts."""
    if isinstance(object_descs, str):
        object_descs = [s.strip() for s in object_descs.split(",") if s.strip()]
    if not object_descs:
        raise ValueError("object_descs must contain at least one description")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ok, frame = cap.read()
    if not ok:
        raise RuntimeError("Video appears empty.")

    pil_first = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # --- ask VLM for every object ---
    init_boxes = []  # (x,y,w,h)
    for desc in object_descs:
        x1, y1, x2, y2 = ask_bbox(pil_first, desc)
        init_boxes.append((x1, y1, x2 - x1, y2 - y1))
        # draw initial bbox for visual check (green)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, desc, (x1, max(0, y1 - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # --- init MultiTracker ---
    multi_tracker = cv2.legacy.MultiTracker_create()
    for box in init_boxes:
        multi_tracker.add(_make_tracker_MOI(tracker_name), frame, box)

    # --- output writer ---
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(out_path, fourcc, fps, (W, H))

    frame_id = 0
    use_gui = show_every and hasattr(cv2, "imshow")
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        success, boxes = multi_tracker.update(frame)
        if success:
            for box, desc in zip(boxes, object_descs):
                x, y, w, h = map(int, box)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, desc, (x, max(0, y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        out.write(frame)
        if use_gui and (frame_id % show_every == 0):
            cv2.imshow("tracking", frame)
            if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
                break
        frame_id += 1

    cap.release()
    out.release()
    if use_gui and hasattr(cv2, "destroyAllWindows"):
        cv2.destroyAllWindows()
    print(f"✔ Finished. Result saved to: {out_path}")


In [12]:
# =========================================================
# 3. Run!  (provide an ABSOLUTE path on the Colab VM)
# =========================================================
# ► If the file is already on the VM (e.g. in /content/videos/demo.mp4)
#   just type that full path.  Otherwise first upload via the left-hand
#   Files pane or `!gdown` / `!wget`.
if __name__ == "__main__":
    # Example on local / Colab
    video_file = "/content/video.mp4"  # absolute path
    targets = ["mouse","hands"]      # list or comma‑separated string
    track_video(video_file, targets, tracker_name="csrt", show_every=0)



✔ Finished. Result saved to: tracked.mp4
